In [2]:
import altair as alt
import numpy as np
import pandas as pd

In [3]:
# summary statistics
fl_opioid = pd.read_csv("/Users/pr158admin/Desktop/FL_WA_DEC10_subsets/FL_DEC10_subset.csv")
wa_opioid = pd.read_csv("/Users/pr158admin/Desktop/FL_WA_DEC10_subsets/WA_DEC10_subset.csv")
#tx = pd.read_csv("/Users/pr158admin/Desktop/Finality Subsets/TX subsetfinalized.csv")

In [23]:
def dataset(df, year, state):
    """make dataset ready to run diff-diff and pre-post for opioids"""
    df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
    groupby_1 = (df.groupby(['BUYER_STATE_x', 'BUYER_COUNTY_x','TRANSACTION_YEAR','population','Deaths',],as_index=False)['opioid_shipment_population_ratio'].sum())
    groupby_1['Total shipment']= groupby_1['population']*groupby_1['opioid_shipment_population_ratio']
    opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())
    opioids_data["death_rate"]= opioids_data["Deaths"]/opioids_data["population"]
    opioids_data["death_rate"]*= 100000
    opioids_data["opioid_shipment_population_ratio"]= opioids_data["Total shipment"]/opioids_data["population"]
    opioids_data["policy"] = 0
    opioids_data.loc[opioids_data["TRANSACTION_YEAR"] > year, "policy"] = 1
    opioids_data["state"] = 0
    opioids_data.loc[opioids_data["BUYER_STATE_x"] == state, "state"] = 1
    return opioids_data


In [24]:
FL = dataset(fl, 2010, "FL")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/1534047806.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [25]:
TX = dataset(tx, 2007, "TX")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/1534047806.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [26]:
WA = dataset(wa, 2012, "WA")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/1534047806.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [27]:
FL.head(10)

,BUYER_STATE_x,TRANSACTION_YEAR,population,Deaths,Total shipment,death_rate,opioid_shipment_population_ratio,policy,state
0,FL,2006,18154693.0,2303.0,7.572175e+06,12.685425,0.417092,0,1
1,FL,2007,18355485.0,2546.0,9.275845e+06,13.870513,0.505345,0,1
2,FL,2008,18514553.0,2660.0,1.207714e+07,14.367077,0.652305,0,1
3,FL,2009,18222420.0,2795.0,1.541160e+07,15.338248,0.845750,0,1
4,FL,2010,18511620.0,2748.0,1.999628e+07,14.844730,1.080202,0,1
5,FL,2011,18688787.0,2667.0,1.518623e+07,14.270589,0.812585,1,1
6,FL,2012,18885152.0,2299.0,9.429126e+06,12.173585,0.499288,1,1
7,FL,2013,19091156.0,2180.0,7.391156e+06,11.418900,0.387151,1,1
8,FL,2014,19361792.0,2448.0,7.102058e+06,12.643458,0.366808,1,1
9,HI,2006,1309622.0,58.0,2.482951e+05,4.428759,0.189593,0,0


In [39]:
#for Flori
summarystat_fl=FL.drop(["Deaths","Total shipment","population","TRANSACTION_YEAR"], axis=1)

In [42]:
summarystat_fl.head()

,BUYER_STATE_x,death_rate,opioid_shipment_population_ratio,policy,state
0,FL,12.685425,0.417092,0,1
1,FL,13.870513,0.505345,0,1
2,FL,14.367077,0.652305,0,1
3,FL,15.338248,0.845750,0,1
4,FL,14.844730,1.080202,0,1


In [47]:
result_FL = (summarystat_fl.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))


/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/3679493063.py:1: FutureWarning: ['BUYER_STATE_x'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  result_FL = (summarystat_fl.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))


In [48]:
result_FL

death_rate                                   \
                   mean     median        min        max   
state policy                                               
0     0        8.356078   8.133476   0.000000  19.509946   
      1       12.219384  10.639025   1.539584  24.913723   
1     0       14.221199  14.367077  12.685425  15.338248   
      1       12.626633  12.408521  11.418900  14.270589   

             opioid_shipment_population_ratio                                
                                         mean    median       min       max  
state policy                                                                 
0     0                              0.356186  0.330365  0.189593  0.621620  
      1                              0.445930  0.432720  0.260427  0.688969  
1     0                              0.700139  0.652305  0.417092  1.080202  
      1                              0.516458  0.443219  0.366808  0.812585

In [50]:
summarystat_tx=TX.drop(["Deaths","Total shipment","population","TRANSACTION_YEAR"], axis=1)
result_TX = (summarystat_tx.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/1070384287.py:2: FutureWarning: ['BUYER_STATE_x'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  result_TX = (summarystat_tx.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))


In [51]:
result_TX

death_rate                                  \
                   mean     median       min        max   
state policy                                              
0     0        8.390618   9.436505  0.000000  14.641554   
      1        9.587561  10.247075  0.000000  16.934207   
1     0        6.913688   6.913688  6.738855   7.088521   
      1        7.389208   7.635940  6.132924   7.986764   

             opioid_shipment_population_ratio                                
                                         mean    median       min       max  
state policy                                                                 
0     0                              0.248010  0.237307  0.111787  0.408516  
      1                              0.343563  0.340392  0.123899  0.604339  
1     0                              0.186408  0.186408  0.171451  0.201365  
      1                              0.218719  0.221937  0.195583  0.235273

In [52]:
summarystat_wa=WA.drop(["Deaths","Total shipment","population","TRANSACTION_YEAR"], axis=1)
result_WA = (summarystat_wa.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_8748/990899825.py:2: FutureWarning: ['BUYER_STATE_x'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  result_WA = (summarystat_wa.groupby(['state','policy',],as_index=False).agg( ['mean', 'median','min', 'max']))


In [53]:
result_WA

death_rate                                   \
                   mean     median        min        max   
state policy                                               
0     0        9.161002   8.299418   2.139288  24.913723   
      1       12.110876  12.250768   2.474741  24.057639   
1     0       12.189354  12.361409  11.217294  12.846241   
      1       11.954125  11.954125  11.892230  12.016020   

             opioid_shipment_population_ratio                                
                                         mean    median       min       max  
state policy                                                                 
0     0                              0.386852  0.379612  0.141574  0.736421  
      1                              0.440441  0.403209  0.209635  0.684118  
1     0                              0.364244  0.376849  0.313929  0.386787  
      1                              0.355953  0.355953  0.355050  0.356856